0.3版本 

In [4]:
# from docutils.nodes import description
from langchain_core.tools import tool, StructuredTool
from pydantic import BaseModel


@tool
def add_number(a: int, b: int) -> int:
    """计算两个整数的和"""
    return a + b


print(f"name = {add_number.name}")  #默认是函数的名称
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")  #默认是函数的说明信息
print(f"return_direct = {add_number.return_direct}")  #默认值是False


name = add_number
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
description = 计算两个整数的和
return_direct = False


1.0版本

In [2]:
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

print(f"name = {search_database.name}")  #默认是函数的名称
print(f"args = {search_database.args}")
print(f"description = {search_database.description}")  #默认是函数的说明信息
print(f"return_direct = {search_database.return_direct}")  #默认值是False

name = search_database
args = {'query': {'title': 'Query', 'type': 'string'}, 'limit': {'default': 10, 'title': 'Limit', 'type': 'integer'}}
description = Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
return_direct = False


In [5]:
@tool("web_search")  # Custom name
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

web_search


使用自定义

In [8]:
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

print(calc.description)

Evaluate mathematical expressions.


0.3  StructuredTool的from_function()的使用


In [23]:

from langchain_core.tools.structured import StructuredTool
from pydantic import BaseModel, Field


class FieldInfo(BaseModel):
    query: str = Field(description="要检索的关键词")


# 声明一个函数
def search_google(query: str):
    return "最后查询的结果"


# 定义一个工具
search02 = StructuredTool.from_function(
    func=search_google,
    name="Search",
    description="查询google搜索引擎，并将结果返回",
    return_direct=True,
    args_schema=FieldInfo
)

print(f"name = {search02.name}")
print(f"args = {search02.args}")
print(f"description = {search02.description}")
print(f"return_direct = {search02.return_direct}")

name = Search
args = {'query': {'description': '要检索的关键词', 'title': 'Query', 'type': 'string'}}
description = 查询google搜索引擎，并将结果返回
return_direct = True


In [10]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

print(f"name = {get_weather.name}")  #默认是函数的名称
print(f"args = {get_weather.args}")
print(f"description = {get_weather.description}")  #默认是函数的说明信息
print(f"return_direct = {get_weather.return_direct}")  #默认值是False
print(f"args = {get_weather.args_schema}")
# print(f"description = {get_weather.units}")  #默认是函数的说明信息
# print(f"return_direct = {get_weather.include_forecast}") 


name = get_weather
args = {'location': {'description': 'City name or coordinates', 'title': 'Location', 'type': 'string'}, 'units': {'default': 'celsius', 'description': 'Temperature unit preference', 'enum': ['celsius', 'fahrenheit'], 'title': 'Units', 'type': 'string'}, 'include_forecast': {'default': False, 'description': 'Include 5-day forecast', 'title': 'Include Forecast', 'type': 'boolean'}}
description = Get current weather and optional forecast.
return_direct = False
args = <class '__main__.WeatherInput'>


JSON模式定义复杂输入

In [18]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

print(f"name = {get_weather.name}")  #默认是函数的名称
print(f"args = {get_weather.args}")
print(f"description = {get_weather.description}")  #默认是函数的说明信息
print(f"return_direct = {get_weather.return_direct}")  #默认值是False
print(f"args = {get_weather.args_schema}")
print(f"args = {get_weather.args_schema['properties']['location']}")

name = get_weather
args = {'location': {'type': 'string'}, 'units': {'type': 'string'}, 'include_forecast': {'type': 'boolean'}}
description = Get current weather and optional forecast.
return_direct = False
args = {'type': 'object', 'properties': {'location': {'type': 'string'}, 'units': {'type': 'string'}, 'include_forecast': {'type': 'boolean'}}, 'required': ['location', 'units', 'include_forecast']}
args = {'type': 'string'}


访问所有运行时信息，集中在一个参数中。只需添加到你的工具签名中，它会自动注入，不会暴露给大型语言模型。ToolRuntimeruntime: ToolRuntime

工具可以通过以下方式访问当前图状态：ToolRuntime

In [21]:
from langchain.tools import tool, ToolRuntime

# Access the current conversation state
@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

print(summarize_conversation)
print(get_user_preference)


False
name='get_user_preference' description='Get a user preference value.' args_schema=<class 'langchain_core.utils.pydantic.get_user_preference'> func=<function get_user_preference at 0x0000019485116340>


In [26]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama



llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

def get_weather(city: str) -> str:
    """获取测试数据"""

    return f"乌云密布 {city}!"

agent = create_agent(
    model=llm,
    tools=[get_weather],
)
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "长沙现在是什么天气?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'id': '62cc0022-0567-42ba-8c73-29d2f632e3de', 'name': 'get_weather', 'args': {'city': '阿尃'}}]
step: tools
content: [{'type': 'text', 'text': '乌云密布 阿尃!'}]
step: model
content: [{'type': 'text', 'text': '根据我的工具调用，长沙当前的天气是乌云密布。'}]


这个例子之后再定义，sqlite

In [41]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
import sqlite3

from langchain_community.utilities import SQLDatabase

db_path = "D:/zy/code/Testdata/LData.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

conn = sqlite3.connect(db_path)  # 连接到 SQLite 数据库
cursor = conn.cursor()

from langchain_community.agent_toolkits import SQLDatabaseToolkit

#  querycondition如果包含多个条件，那么每个字段需要单独用[],例如转速 > 1300 AND 扭矩 > 1500应该用 [转速] > 1300 AND [扭矩] > 1500

class TestDataQuery(BaseModel):
    """获取测试数据,格式是 SELECT COUNT(*) FROM table where querycondition"""
    table: str = Field(description="数据库表的名称")
    querycondition: str = Field(description="查询条件")
    


llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

@tool(args_schema=TestDataQuery)
def get_testdata(table: str, querycondition: str) -> str:
    """获取测试数据,格式是 SELECT COUNT(*) FROM table where querycondition"""
    res = db.run("SELECT COUNT(*) FROM table where querycondition")
    
    return f"res {res}!"

system_prompt = '你是一个数据库查询助手, 【20240315_外特性_1】表是数据库中存储外特性数据的表'

agent = create_agent(
    model=llm,
    tools=[get_testdata],    #.append(tools),
    system_prompt=system_prompt,
)

question = "存储外特性数据的表的数据是什么，请注意，sqllite的表需要使用[]包起来进行查询,只需要显示转速字段中值大于1300和扭矩字段中大于1500的记录"

for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": question}]},
    # stream_mode="updates",
    stream_mode="values",
):
    # for step, data in chunk.items():
    #     print(f"step: {step}")
    #     print(f"content: {data['messages'][-1].content_blocks}")
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

存储外特性数据的表的数据是什么，请注意，sqllite的表需要使用[]包起来进行查询,只需要显示转速字段中值大于1300和扭矩字段中大于1500的记录
================================== Ai Message ==================================
Tool Calls:
  get_testdata (3928d3a4-723b-4f0a-b291-af4979a09b26)
 Call ID: 3928d3a4-723b-4f0a-b291-af4979a09b26
  Args:
    querycondition: 转速 > 1300 AND 扭矩 > 1500
    table: ['外特性数据表']
================================= Tool Message =================================
Name: get_testdata

Error invoking tool 'get_testdata' with kwargs {'querycondition': '转速 > 1300 AND 扭矩 > 1500', 'table': ['外特性数据表']} with error:
 table: Input should be a valid string
 Please fix the error and try again.
================================== Ai Message ==================================
Tool Calls:
  get_testdata (75ba1189-ff6b-476e-a456-f3755d140340)
 Call ID: 75ba1189-ff6b-476e-a456-f3755d140340
  Args:
    querycondition: 转速 > 1300 AND 扭矩 > 1500
    table: [外特性数据表]


OperationalError: (sqlite3.OperationalError) near "table": syntax error
[SQL: SELECT COUNT(*) FROM table where querycondition]
(Background on this error at: https://sqlalche.me/e/20/e3q8)